# Part 3: Chains and Runnables in LangChain

## Chains: A Look Back

Imagine LangChain in its early days, when developers were just starting to figure out how to build applications around Large Language Models (LLMs). There was a clear need to string together different operations. You'd want to take some user input, pass it through a prompt, then send that to an LLM, and maybe even process the LLM's output further. This is precisely what **Chains** were designed to do.

### What Chains Were and Their Original Purpose

In essence, **Chains were sequences of calls, or "links," that processed data in a predefined order.** Each link in the chain would take an input, perform an operation, and then pass its output as the input to the next link. Think of it like an assembly line, where each station performs a specific task.

The original purpose of Chains was to simplify the development of multi-step LLM applications. Instead of manually managing the input and output of each component, you could define a chain, and LangChain would handle the flow for you.

### Simple Chain Examples

Let's look at a very basic example of what a chain might have looked like. Remember, we're not going to run this code as Chains are largely deprecated, but it helps to understand the concept.

```python
# This is conceptual code to illustrate old Chains
# Do NOT run this code as it uses deprecated patterns

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain # Deprecated import

# 1. Define a Prompt Template
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

# 2. Define the LLM (hypothetically)
llm = OpenAI(temperature=0.7)

# 3. Create an LLMChain
# llm_chain = LLMChain(prompt=prompt, llm=llm) # This is how it used to be

# To use it:
# response = llm_chain.run("colorful socks")
# print(response)
```

In this conceptual example, the `LLMChain` took a prompt and an LLM, effectively chaining them together. You'd give it the product, it would format the prompt, send it to the LLM, and return the LLM's response.

You could even create more complex chains, for instance, by adding a simple parsing step:

```python
# This is conceptual code to illustrate old Chains
# Do NOT run this code as it uses deprecated patterns

# from langchain.chains import SimpleSequentialChain # Deprecated import

# prompt1 = PromptTemplate(...) # First prompt
# llm1 = OpenAI(...) # First LLM
# chain1 = LLMChain(prompt=prompt1, llm=llm1)

# prompt2 = PromptTemplate(...) # Second prompt
# llm2 = OpenAI(...) # Second LLM
# chain2 = LLMChain(prompt=prompt2, llm=llm2)

# overall_chain = SimpleSequentialChain(chains=[chain1, chain2]) # Chaining them
# response = overall_chain.run("some input")
```

Here, `SimpleSequentialChain` allowed you to run multiple chains in sequence, where the output of one became the input of the next.

### Key Limitations that Led to Their Replacement

While Chains were a good start, they had some significant limitations that became apparent as LangChain grew and the complexity of LLM applications increased:

1.  **Lack of Flexibility:** Chains were often rigid. If you wanted to do something that wasn't a simple sequential flow, like branching logic, parallel execution, or handling multiple inputs/outputs, it became cumbersome or impossible.
2.  **Limited Error Handling and Debugging:** Debugging complex chains could be challenging. It was often hard to pinpoint where an error occurred or to inspect intermediate results.
3.  **No Streaming or Async Support:** As LLMs became more responsive, the need for streaming outputs (like seeing words appear one by one) and asynchronous execution (running multiple things at once without blocking) became crucial. Chains weren't built with this in mind.
4.  **Poor Compositionality:** While they introduced the *concept* of composition, the way Chains were designed often led to a hierarchical, nested structure that was hard to reason about and modify. It wasn't always clear how to compose different types of operations effectively.
5.  **Performance Issues:** Without native support for concurrent execution, performance could suffer in scenarios where multiple LLM calls or other operations could theoretically run in parallel.

### How Chains Introduced the Concept of Component Composition

Despite their limitations, Chains were instrumental in introducing a fundamental idea that still underpins LangChain: **component composition**. They showed us the power of breaking down complex tasks into smaller, manageable pieces (like prompts, LLMs, parsers) and then assembling those pieces into a larger workflow. This modularity is key to building scalable and maintainable LLM applications.

### This is Where Runnables Come In...

The LangChain team recognized these limitations and completely reimagined how components should interact. They wanted a system that was more flexible, more powerful, and inherently supported modern application development patterns like streaming and asynchronicity.

And that, my friends, is why **Runnables** were born\! They are the evolution of component composition in LangChain, designed to address all the shortcomings of Chains and provide a robust framework for building almost any LLM application imaginable.

-----

## Runnables: The Core of LangChain Expression Language (LCEL)

If Chains were the initial blueprint for connecting components, **Runnables** are the robust, extensible, and high-performance engineering marvel that powers LangChain today. They are the building blocks of what's known as the **LangChain Expression Language (LCEL)**, and understanding them is crucial for building anything significant with LangChain.

### Foundational Concepts

#### What Runnables Are and the Problem They Solve

At their heart, **Runnables are a standardized interface for interacting with any component in LangChain.** Whether it's a prompt, an LLM, an output parser, a custom function, or even another complex sequence of operations, if it's a Runnable, you know how to interact with it.

The problem they solve is **interoperability and composability**. Before Runnables, every component might have had a slightly different way of being called, or of handling inputs and outputs. This made it difficult to combine them flexibly. Runnables provide a unified API, meaning you can easily connect any Runnable to any other Runnable, much like LEGO bricks snapping together.

#### The Runnable Interface and Its Core Methods

The power of Runnables comes from their shared interface. Every object that implements the `Runnable` interface guarantees it has certain methods, making them predictable and easy to work with.

The most important core methods you'll use constantly are:

  * `invoke()`: This is the primary method for calling a Runnable. It takes a single input and returns a single output. It's synchronous, meaning it will block execution until the result is ready.
  * `stream()`: This method allows you to get outputs incrementally as they are generated. This is incredibly useful for LLMs, where you want to display the response word-by-word to the user rather than waiting for the entire response. It returns an iterator.
  * `batch()`: This method allows you to run a Runnable on multiple inputs concurrently or in parallel. This is great for performance optimization when you have many independent requests. It takes a list of inputs and returns a list of outputs.
  * `ainvoke()`, `astream()`, `abatch()`: These are the asynchronous versions of the above methods. They allow you to run operations without blocking the main program thread, which is essential for building responsive web applications or high-throughput services. We won't dive deep into async in this lecture, but be aware they exist for advanced use cases.

**Pro Tip:** Think of `invoke()` as the fundamental way to run a Runnable when you just need the final result. `stream()` is for when you want to see the progress, and `batch()` is for when you have a lot of things to process at once.

#### Why Runnables Replaced Chains

Runnables didn't just replace Chains; they completely redefined how we build with LangChain. Here's a summary of why they won out:

  * **Unified API:** Chains had various types (`LLMChain`, `SimpleSequentialChain`, `SequentialChain`, etc.), each with slightly different interfaces. Runnables offer a single, consistent way to interact with any component.
  * **Built-in Streaming & Async:** From the ground up, Runnables were designed to support streaming (think real-time LLM output) and asynchronous operations, which are critical for modern, performant applications.
  * **Enhanced Composability (LCEL):** Runnables are the foundation of LCEL, which provides a powerful, intuitive syntax (using the `|` operator, like Unix pipes) for composing complex workflows. This is far more flexible than the rigid structures of old Chains.
  * **Better Error Handling & Debugging:** Because of their modular nature and consistent interface, it's easier to inspect intermediate steps and debug issues within a Runnable sequence.
  * **Performance:** LCEL, built on Runnables, offers automatic parallelization for independent steps when possible, leading to significant performance gains.

#### The Philosophy Behind the Runnable Architecture with Code

The core philosophy behind Runnables and LCEL is that **every step in your LLM application should be treated as a first-class citizen that can be composed with others.** This leads to highly modular, reusable, and testable code.

Let's illustrate this with a simple example. We'll set up our environment first.

```python
# First, let's make sure we have the necessary libraries installed
# If you don't have them, run:
# pip install -q langchain-core langchain-community langchain-openai

import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Set your OpenAI API key (replace with your actual key or environment variable)
# It's best practice to set this as an environment variable (e.g., export OPENAI_API_KEY='your_key_here')
# For demonstration, we'll set it directly here, but use environment variables in production.
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY" # Uncomment and replace!

# Let's assume you have an API key set up as an environment variable called OPENAI_API_KEY
# If not, you'll need to set it up before running this code.

# Initialize our LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Initialize an output parser (very common to convert LLM output to a simple string)
parser = StrOutputParser()
```

Now, let's see the Runnable philosophy in action. We'll create a simple chain to generate a company name.

```python
# Define our prompt template using the ChatPromptTemplate (a Runnable!)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that generates creative company names."),
    ("user", "What is a good name for a company that makes {product}?")
])

# Now, let's combine them using the | operator, which is the core of LCEL.
# This creates a RunnableSequence under the hood.
company_name_generator = prompt | llm | parser

# Now, let's invoke it!
response = company_name_generator.invoke({"product": "eco-friendly water bottles"})
print(f"Generated Company Name: {response}")

# Let's try another one
response = company_name_generator.invoke({"product": "AI-powered coffee makers"})
print(f"Generated Company Name: {response}")
```

See how elegant that is? `prompt`, `llm`, and `parser` are all `Runnable` objects. The `|` operator takes the output of the component on the left and passes it as input to the component on the right. This is the **LangChain Expression Language (LCEL)** in action, and it's built entirely on the `Runnable` interface.

**Key Takeaway:**
The `|` (pipe) operator is your best friend when working with Runnables. It allows you to chain Runnables together in a highly readable and intuitive way, forming powerful data pipelines.

-----

## Core Runnable Types

Now that you understand the foundational concepts, let's explore the most important built-in Runnable types you'll encounter and use daily. These are the workhorses of LCEL, allowing you to build incredibly flexible and powerful workflows.

### RunnableLambda: Custom Functions, Data Transformation, Business Logic

Imagine you have a piece of custom logic that you want to integrate seamlessly into your LangChain workflow. Maybe you need to preprocess user input, format an LLM's output in a specific way, or apply some business rules. That's where `RunnableLambda` comes in\!

`RunnableLambda` allows you to wrap any Python function, making it a `Runnable`. This means you can use your custom functions within an LCEL chain, just like any other LangChain component.

#### Explanation and Examples

A `RunnableLambda` takes a function as its input. This function should typically accept one argument (the input from the previous step) and return one output (to be passed to the next step).

```python
from langchain_core.runnables import RunnableLambda

# Example 1: Simple text transformation
def to_uppercase(text: str) -> str:
    """Converts input text to uppercase."""
    return text.upper()

# Create a RunnableLambda from our function
uppercase_runnable = RunnableLambda(to_uppercase)

# Let's invoke it
print(f"Original: 'hello world', Uppercase: {uppercase_runnable.invoke('hello world')}")

# Now, let's integrate it into a chain
# Reuse our existing LLM and parser
# company_name_generator = prompt | llm | parser # from previous example

# Let's create a prompt that asks for a product, then capitalize it, then send to LLM
capitalizer_chain = RunnableLambda(lambda x: {"product": x.upper()}) | prompt | llm | parser

print("\n--- Example 1: Capitalizing product name before LLM ---")
response = capitalizer_chain.invoke("eco-friendly pens")
print(f"Generated Name (capitalized input): {response}")

# Example 2: More complex data transformation (e.g., preparing input for prompt)
def prepare_product_info(product_name: str) -> dict:
    """Formats a product name into a dictionary suitable for a prompt."""
    return {"product": f"high-quality, innovative {product_name}"}

prepare_input_runnable = RunnableLambda(prepare_product_info)

# Now, let's chain it
enhanced_generator = prepare_input_runnable | prompt | llm | parser

print("\n--- Example 2: Enhancing product description ---")
response = enhanced_generator.invoke("smartwatch")
print(f"Generated Name (enhanced input): {response}")

# Example 3: Post-processing LLM output
def add_trademark_symbol(name: str) -> str:
    """Adds a TM symbol to the generated company name."""
    return f"{name}™"

add_trademark_runnable = RunnableLambda(add_trademark_symbol)

# Let's modify our original company name generator
final_company_name_generator = prompt | llm | parser | add_trademark_runnable

print("\n--- Example 3: Adding a trademark symbol to output ---")
response = final_company_name_generator.invoke({"product": "sustainable fashion"})
print(f"Final Company Name: {response}")
```

`RunnableLambda` is incredibly versatile for injecting any custom Python logic into your LCEL chains. It's perfect for data cleaning, reformatting, validation, or applying any specific business rule.

**Common Pitfall:** Ensure your `RunnableLambda` function's input and output types match what the preceding and succeeding Runnables expect. If you're passing a string but the next component expects a dictionary, you'll get an error.

-----

### RunnablePassthrough: Data Flow Control, Branching, Selective Passing

Sometimes, you need to pass an input through a chain **without modifying it**, or you need to inject additional information into the chain at a specific point. `RunnablePassthrough` is your go-to for these scenarios. It's essentially a no-op (no operation) that just forwards its input.

#### Explanation and Examples

The simplest use of `RunnablePassthrough` is to just pass data along. But its true power comes when combined with other Runnables, especially for managing inputs in more complex scenarios.

```python
from langchain_core.runnables import RunnablePassthrough

# Example 1: Simply passing data through
passthrough_chain = RunnablePassthrough() | RunnableLambda(lambda x: f"Processed: {x}")

print("\n--- Example 1: Simple Passthrough ---")
print(passthrough_chain.invoke("initial data"))

# Example 2: Injecting additional context into a chain
# Imagine your prompt needs more than just one input, but you only have one initial input.
# You can use .assign() or .partial() with RunnablePassthrough for this.

# Let's say our prompt now needs both 'product' and 'industry'
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative naming assistant."),
    ("user", "Suggest a unique name for a company in the {industry} that makes {product}.")
])

# We only have 'product' as input initially. We want to add 'industry'.
# .assign() is very powerful for adding/modifying keys in a dictionary input.
# RunnablePassthrough.assign() takes another Runnable or dictionary as an argument.
# Here, we're using a lambda to define the new 'industry' key.

print("\n--- Example 2: Injecting context with RunnablePassthrough.assign() ---")
contextual_company_generator = (
    {"product": RunnablePassthrough()} # Takes the input, puts it under 'product'
    | RunnablePassthrough.assign(industry=RunnableLambda(lambda x: "tech" if "AI" in x else "general goods")) # Dynamically adds 'industry'
    | contextual_prompt
    | llm
    | parser
)

print("Generated Name (tech industry):", contextual_company_generator.invoke("AI-powered drones"))
print("Generated Name (general goods):", contextual_company_generator.invoke("handmade soaps"))

# Example 3: Passing input to multiple branches simultaneously (implicitly useful with RunnableParallel)
# While RunnablePassthrough itself doesn't branch, it's often used when you have
# an input that needs to be consumed by multiple subsequent Runnables in parallel.
# We'll see this more clearly with RunnableParallel, but it's important to know
# that if you pass input directly to a Runnable that expects multiple inputs (like a RunnableParallel),
# that input will be passed to *all* branches unless specified otherwise.
```

`RunnablePassthrough` is crucial for manipulating the flow of data, especially when you need to ensure certain inputs are available at different stages of your chain, or when you're preparing inputs for more complex structures like `RunnableParallel` or `RunnableBranch`.

**Pro Tip:** `RunnablePassthrough.assign()` is one of the most useful methods for managing and enriching your input dictionary as it flows through a chain. You can add new keys, or even overwrite existing ones based on previous steps.

-----

### RunnableParallel: Concurrent Execution, Data Aggregation, Performance Optimization

What if you have multiple operations that don't depend on each other and can run at the same time? Running them sequentially would waste time. `RunnableParallel` solves this by allowing you to **execute multiple Runnables concurrently**, collecting all their results. This is a powerful tool for performance optimization and for gathering diverse pieces of information in parallel.

#### Explanation and Examples

`RunnableParallel` takes a dictionary of Runnables. Each key in the dictionary becomes a key in the output, and its value is the result of running the corresponding Runnable. All Runnables within a `RunnableParallel` are executed at the same time.

```python
from langchain_core.runnables import RunnableParallel

# Example 1: Fetching multiple pieces of information about a product
# Imagine we want a company name AND a slogan for the same product.

name_generator = (
    ChatPromptTemplate.from_template("What's a good company name for {product}?")
    | llm
    | parser
)

slogan_generator = (
    ChatPromptTemplate.from_template("Generate a catchy slogan for a company that makes {product}.")
    | llm
    | parser
)

# Now, let's run them in parallel
parallel_info_generator = RunnableParallel(
    company_name=name_generator,
    slogan=slogan_generator
)

print("\n--- Example 1: Parallel Name and Slogan Generation ---")
product_input = {"product": "organic cat food"}
results = parallel_info_generator.invoke(product_input)
print(f"Results for '{product_input['product']}':")
print(f"  Company Name: {results['company_name']}")
print(f"  Slogan: {results['slogan']}")

# Note: The input to RunnableParallel is passed to *each* of its internal Runnables.
# In this case, 'product_input' is given to both name_generator and slogan_generator.

# Example 2: More complex parallelization with a shared LLM
# Let's say we want to generate a short description and a list of features for a product.

description_prompt = ChatPromptTemplate.from_template(
    "Write a short, engaging description for a company that makes {product}. Keep it under 20 words."
)
features_prompt = ChatPromptTemplate.from_template(
    "List 3 key features for a company that specializes in {product}. List them as bullet points."
)

product_details_generator = RunnableParallel(
    description=(description_prompt | llm | parser),
    features=(features_prompt | llm | parser)
)

print("\n--- Example 2: Parallel Description and Features Generation ---")
product_input_2 = {"product": "AI-powered smart homes"}
results_2 = product_details_generator.invoke(product_input_2)
print(f"Results for '{product_input_2['product']}':")
print(f"  Description: {results_2['description']}")
print(f"  Features: {results_2['features']}")

# Example 3: Handling multiple inputs with RunnablePassthrough for complex scenarios
# What if you want to pass *different* inputs to parallel branches?
# You'd typically use RunnablePassthrough to feed the right data.

# This scenario is less common for simple 'invoke' but becomes very powerful
# when the input to the overall chain contains multiple pieces of data.
# For instance, if the input is `{"topic": "AI", "style": "humorous"}`,
# you could route 'topic' to one branch and 'style' to another.

# For now, just understand that the overall input to RunnableParallel is
# distributed to each sub-runnable. If a sub-runnable needs specific keys,
# ensure they are present in the dictionary input passed to RunnableParallel.
```

`RunnableParallel` is invaluable for optimizing performance by allowing independent tasks to run simultaneously. It's also great for gathering disparate pieces of information that all originate from a single input, providing a consolidated output.

**Performance Consideration:** `RunnableParallel` will run its internal Runnables concurrently. If these Runnables involve network calls (like to an LLM API), this can significantly reduce the total execution time compared to running them sequentially.

-----

### RunnableSequence: Linear Workflows, Step-by-Step Processing

This is arguably the most common and intuitive way to chain Runnables. `RunnableSequence` (which is implicitly created when you use the `|` pipe operator) allows you to define a **linear flow of operations**, where the output of one step becomes the input of the next.

#### Explanation and Examples

You've already been using `RunnableSequence` without explicitly calling it\! Every time you do `runnable1 | runnable2 | runnable3`, you are creating a `RunnableSequence`. It's foundational for building pipelines.

```python
from langchain_core.runnables import RunnableSequence

# We'll re-use our prompt, llm, and parser

# Example 1: Basic linear flow (same as using | operator)
# Define a RunnableSequence explicitly (though usually you just use | )
simple_sequence = RunnableSequence(prompt, llm, parser)

print("\n--- Example 1: Basic Linear Flow (explicit RunnableSequence) ---")
response = simple_sequence.invoke({"product": "sustainable shoes"})
print(f"Generated Name: {response}")

# Example 2: Preprocessing and post-processing in a sequence
# Let's say we want to validate the input product name, then generate the name, then format it.

def validate_product_name(product_name: str) -> str:
    """Ensures the product name is not empty."""
    if not product_name or len(product_name) < 2:
        raise ValueError("Product name cannot be empty or too short.")
    return product_name

def format_company_name_output(name: str) -> str:
    """Adds a friendly message to the generated name."""
    return f"Great choice! Your new company name is: '{name}'"

validation_step = RunnableLambda(validate_product_name)
formatting_step = RunnableLambda(format_company_name_output)

# Building a more sophisticated sequence
full_pipeline = (
    validation_step
    | {"product": RunnablePassthrough()} # Ensure 'product' key is present for the prompt
    | prompt
    | llm
    | parser
    | formatting_step
)

print("\n--- Example 2: Preprocessing and Post-processing in a Sequence ---")
try:
    print(full_pipeline.invoke("innovative headphones"))
    # This will raise an error due to validation
    print(full_pipeline.invoke(""))
except ValueError as e:
    print(f"Error caught as expected: {e}")
```

`RunnableSequence` is the backbone of most LangChain applications. It allows you to define clear, step-by-step processes for handling inputs, interacting with LLMs, and processing outputs.

**Best Practice:** Keep individual steps in your `RunnableSequence` focused on a single responsibility. This makes your code more readable, reusable, and easier to debug.

-----

### RunnableBranch: Conditional Logic, Dynamic Routing, Decision Trees

What if your workflow needs to make a decision based on the input? For example, if the user asks for a simple fact, use one LLM prompt; if they ask for creative writing, use another. `RunnableBranch` is designed for exactly this: **conditional execution**. It allows you to route the flow of data to different Runnables based on certain conditions.

#### Explanation and Examples

`RunnableBranch` takes a list of `(condition, runnable_if_true)` tuples, and an optional `default_runnable`. It evaluates each `condition` function in order. The first condition that returns `True` will cause its associated `runnable_if_true` to be executed. If no condition is met, the `default_runnable` is executed.

The `condition` should be a function that takes the input to the branch and returns `True` or `False`.

```python
from langchain_core.runnables import RunnableBranch

# Let's define some different prompts/LLM chains for different types of questions.
# We'll use the same LLM for simplicity.

# 1. Simple Fact Question Prompt
fact_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers factual questions concisely."),
    ("user", "Answer this question briefly: {question}")
])
fact_chain = fact_prompt | llm | parser

# 2. Creative Writing Prompt
creative_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative writer. Elaborate on the following topic."),
    ("user", "Write a short paragraph about: {topic}")
])
creative_chain = creative_prompt | llm | parser

# 3. Default/Fallback Prompt
default_prompt = ChatPromptTemplate.from_messages([
    ("system", "I'm not sure how to answer that. Please rephrase your query."),
    ("user", "{input}")
])
default_chain = default_prompt | llm | parser

# Now, define the conditions
def is_fact_question(input_dict: dict) -> bool:
    """Checks if the input looks like a factual question."""
    question = input_dict.get("input", "").lower()
    return any(word in question for word in ["who", "what", "where", "when", "why", "how", "is", "are"])

def is_creative_request(input_dict: dict) -> bool:
    """Checks if the input looks like a creative writing request."""
    request = input_dict.get("input", "").lower()
    return any(word in request for word in ["write", "describe", "story", "poem", "essay"])

# Create the RunnableBranch
# We need to map the incoming 'input' to the expected 'question' or 'topic' keys
# within each branch's prompt.
branch = RunnableBranch(
    (
        RunnableLambda(is_fact_question),
        {"question": RunnablePassthrough()} | fact_chain # If factual, map 'input' to 'question'
    ),
    (
        RunnableLambda(is_creative_request),
        {"topic": RunnablePassthrough()} | creative_chain # If creative, map 'input' to 'topic'
    ),
    default_chain # Fallback if no condition matches
)

print("\n--- Example: Conditional Routing with RunnableBranch ---")

print("Asking a factual question:")
response_fact = branch.invoke({"input": "What is the capital of France?"})
print(f"Response (factual): {response_fact}")

print("\nAsking for creative writing:")
response_creative = branch.invoke({"input": "Write a short story about a talking cat."})
print(f"Response (creative): {response_creative}")

print("\nAsking an unknown type of question:")
response_default = branch.invoke({"input": "Tell me something random."})
print(f"Response (default): {response_default}")
```

`RunnableBranch` is powerful for building dynamic applications that can adapt their behavior based on the user's input or other runtime conditions. It's essential for creating more intelligent and nuanced LLM applications.

**Important Note:** The input passed to the `RunnableBranch` itself is passed to each condition function. The output of the `RunnableBranch` will be the output of the chosen branch's Runnable. Pay attention to how your branches expect their input. In the example above, we use `{"question": RunnablePassthrough()}` to map the generic `input` key from the branch to the specific `question` key expected by the `fact_prompt`.

-----

### RunnableMap: Dictionary-like Operations, Structured Data Handling

`RunnableMap` is a specialized Runnable that is incredibly useful when you need to **create a dictionary of inputs for a subsequent Runnable**, where each key in the dictionary is populated by the output of a specific Runnable. It's like building a structured output from multiple sources or computations.

#### Explanation and Examples

`RunnableMap` takes a dictionary where keys are the desired output keys and values are Runnables. It executes all the Runnables within the map in parallel and then combines their outputs into a single dictionary.

```python
from langchain_core.runnables import RunnableMap

# Example 1: Combining multiple static values and a dynamic value
# Imagine you want to prepare a dictionary for a prompt that needs 'city', 'country', and a 'current_date'.

# Let's get the current date using a lambda
get_current_date = RunnableLambda(lambda x: "July 1, 2025") # Simplified for example

# Create a RunnableMap to prepare the input dictionary for a hypothetical prompt
input_preparer = RunnableMap(
    city=RunnableLambda(lambda x: "Abuja"),
    country=RunnableLambda(lambda x: "Nigeria"),
    date=get_current_date
)

# Now, let's "invoke" it. The input 'x' to the lambdas above is the input to the RunnableMap.
# Since our lambdas don't use 'x', we can just pass an empty dict or any input.
print("\n--- Example 1: Creating a structured input dictionary ---")
prepared_input = input_preparer.invoke({}) # We don't need any input for this specific map
print(f"Prepared Input: {prepared_input}")

# Example 2: Preparing inputs for a prompt that requires multiple distinct values
# Let's say we have a prompt that needs 'query' and 'context'.
# And our initial input is just a 'question'. We need to transform it.

# Hypothetical context retriever (for now, just a lambda)
# In a real RAG system, this would be a complex retrieval step!
def retrieve_context(question: str) -> str:
    """Simulates retrieving context based on a question."""
    if "Python" in question:
        return "Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability..."
    return "No specific context found."

context_retriever = RunnableLambda(retrieve_context)

# Our prompt now expects 'query' and 'context'
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's query based on the provided context. If context is insufficient, state that."),
    ("user", "Context: {context}\nQuery: {query}")
])

# Use RunnableMap to create the dictionary expected by qa_prompt
qa_chain = RunnableMap(
    query=RunnablePassthrough(), # The original input (question) becomes the 'query'
    context=context_retriever # The output of context_retriever becomes the 'context'
) | qa_prompt | llm | parser

print("\n--- Example 2: Preparing structured input for a QA prompt ---")
question1 = "What is Python?"
print(f"\nQuestion: {question1}")
response1 = qa_chain.invoke(question1)
print(f"Answer: {response1}")

question2 = "Tell me about cars."
print(f"\nQuestion: {question2}")
response2 = qa_chain.invoke(question2)
print(f"Answer: {response2}")
```

`RunnableMap` is a powerful construct for creating structured data from disparate sources, often used to prepare the exact dictionary inputs expected by prompts or other complex Runnables. It parallelizes the execution of its internal Runnables, which is a performance benefit.

**Key Use Case:** `RunnableMap` is frequently used right before a `ChatPromptTemplate` or any Runnable that expects a dictionary with specific keys. You use it to assemble that dictionary from various upstream sources.

-----

### Custom Runnables: Building Your Own, Inheritance Patterns, Best Practices

While LangChain provides a rich set of built-in Runnables, there will be times when you need to create your own custom logic that integrates seamlessly into an LCEL chain. This is where creating **Custom Runnables** comes in handy. It allows you to encapsulate complex logic, make it reusable, and leverage the full power of the Runnable interface (streaming, async, batching).

#### Explanation and Examples

To create a custom Runnable, you typically inherit from `langchain_core.runnables.Runnable`. You then implement the `_call` or `_invoke` (for synchronous), `_stream` (for streaming), and `_batch` (for batching) methods, depending on the functionality you need. For simplicity, we'll focus on `_invoke` for now, but be aware of the others for more advanced use cases.

When inheriting from `Runnable`, your custom class gets all the methods like `invoke()`, `stream()`, `batch()`, and crucially, it can be seamlessly used with the `|` operator in LCEL.

```python
from langchain_core.runnables import Runnable, chain

# Example 1: A simple custom Runnable for text processing
class LengthCalculator(Runnable):
    """A custom Runnable that calculates the length of a string."""

    def invoke(self, input: str, config=None) -> int:
        """Synchronously calculates the length of the input string."""
        print(f"DEBUG: LengthCalculator received input: '{input}'")
        return len(input)

    # You would also implement _stream and _batch for full Runnable compliance
    # For now, we'll keep it simple.

# Use our custom Runnable in a chain
length_pipeline = (
    RunnablePassthrough() # Input is passed through
    | LengthCalculator()  # Our custom Runnable
    | RunnableLambda(lambda length: f"The text has {length} characters.") # Format output
)

print("\n--- Example 1: Custom LengthCalculator Runnable ---")
print(length_pipeline.invoke("Hello LangChain!"))
print(length_pipeline.invoke("Short"))

# Example 2: A custom Runnable that simulates an external API call
class ProductRecommender(Runnable):
    """
    A custom Runnable that simulates a product recommendation API call.
    It takes a product type and returns a recommended product.
    """
    def invoke(self, product_type: str, config=None) -> str:
        print(f"DEBUG: Recommender received request for: '{product_type}'")
        # Simulate network delay
        import time
        time.sleep(0.1)
        if "electronics" in product_type.lower():
            return "Latest Smartphone Model X"
        elif "books" in product_type.lower():
            return "Classic Sci-Fi Novel"
        return "Generic Recommended Item"

# Let's create a chain that takes a user query, identifies a product type,
# then uses our recommender, and finally presents the recommendation.

# Simplified product type extractor using LLM (for demonstration)
product_type_extractor_prompt = ChatPromptTemplate.from_template(
    "Identify the main product type in the following query: '{query}'. Respond with just the product type (e.g., 'electronics', 'books')."
)
product_type_extractor = product_type_extractor_prompt | llm | parser

# Now, integrate our custom ProductRecommender
recommendation_pipeline = (
    {"query": RunnablePassthrough()} # Takes the original query as input
    | RunnableMap(
        product_type=product_type_extractor, # Extract product type in parallel (or sequentially, depending on structure)
        original_query=RunnablePassthrough() # Keep original query if needed later
    )
    | RunnableLambda(lambda x: x['product_type']) # Just pass the extracted product_type to the recommender
    | ProductRecommender()
    | RunnableLambda(lambda rec: f"Based on your query, we recommend: {rec}")
)

print("\n--- Example 2: Custom ProductRecommender Runnable ---")
print(recommendation_pipeline.invoke("I'm looking for a new smartphone."))
print(recommendation_pipeline.invoke("Can you suggest a good novel?"))
print(recommendation_pipeline.invoke("What about kitchen appliances?"))

# Decorator-based approach for simpler functions
# For simpler custom logic that doesn't need to manage state or complex methods,
# you can use the @chain decorator, which automatically wraps a function into a Runnable.

@chain
def reverse_string(text: str) -> str:
    """Reverses the input string."""
    return text[::-1]

reverse_pipeline = RunnablePassthrough() | reverse_string

print("\n--- Example 3: Custom Runnable with @chain decorator ---")
print(reverse_pipeline.invoke("hello world"))

```

#### Inheritance Patterns and Best Practices

  * **Inherit from `Runnable`:** For most custom components, inheriting directly from `langchain_core.runnables.Runnable` is the way to go.
  * **Implement `invoke()` and/or `stream()`/`batch()`:** Decide which methods you need to support. For simple synchronous operations, `invoke()` is sufficient. For real-time applications or performance, consider `stream()` and `batch()`.
  * **Handle `config`:** All `Runnable` methods (like `invoke`, `stream`, `batch`) receive a `config` argument. This dictionary can contain runtime configuration like callbacks, tags, or a unique run ID. While you might not use it immediately, it's good practice to include `**kwargs` or `config=None` in your method signatures.
  * **Input/Output Consistency:** Ensure your custom Runnable consistently takes a single input (of a specific type) and produces a single output (of a specific type). This is crucial for seamless chaining.
  * **Encapsulation:** Custom Runnables are excellent for encapsulating complex business logic, external API calls, or specific data transformations, keeping your main LCEL chains clean and readable.

**Key Takeaway:**
Custom Runnables are your pathway to extending LangChain with your unique business logic and integrating it perfectly into the LCEL framework. They are fundamental for building sophisticated and tailored LLM applications.

-----

## Troubleshooting Common Issues

Working with Runnables is generally smooth, but like any programming, you might hit a snag. Here are some common issues and how to troubleshoot them:

1.  **Input/Output Type Mismatch:**

      * **Symptom:** You get errors like "TypeError: sequence item 0: expected str instance, dict found" or "ValueError: Input to prompt must be a dictionary with keys...".
      * **Cause:** One Runnable is outputting a type (e.g., a string) that the next Runnable isn't expecting (e.g., a dictionary), or vice-versa.
      * **Solution:** Carefully inspect the input and output types of each Runnable in your chain. Use `RunnableLambda` for transformations or `RunnablePassthrough.assign()`/`RunnableMap` to reshape dictionaries. Print intermediate outputs using `RunnableLambda(lambda x: print(f"Intermediate: {x}") or x)`.

2.  **Missing Dictionary Keys:**

      * **Symptom:** Your `ChatPromptTemplate` complains about missing input variables (e.g., "Input variables 'product' are not found").
      * **Cause:** The dictionary being passed to your prompt (or any other Runnable expecting specific keys) doesn't contain all the required keys.
      * **Solution:** Use `RunnableMap` or `RunnablePassthrough.assign()` upstream to ensure all necessary keys are present in the dictionary before it reaches the component that needs them.

3.  **Unexpected Output Format from LLM:**

      * **Symptom:** Your parser or subsequent logic fails because the LLM's raw output isn't in the expected format (e.g., not JSON, or not a simple string).
      * **Cause:** The LLM's response deviates from what you instructed in the prompt, or your prompt wasn't specific enough.
      * **Solution:**
          * **Refine your prompt:** Be extremely explicit in your prompt about the desired output format (e.g., "Respond ONLY with a JSON object...", "Your answer should be a single word...").
          * **Use Output Parsers:** LangChain provides various `OutputParser` classes that can help (like `StrOutputParser`, `JsonOutputParser`). Integrate them into your chain.
          * **Use `RunnableLambda` for custom parsing:** If standard parsers don't work, write a `RunnableLambda` to parse and clean the LLM's output.

4.  **Debugging Chains:**

      * **Tip:** When a chain is not behaving as expected, break it down. Run each Runnable independently, passing it the expected input, and inspect its output.
      * **Tip:** Insert `RunnableLambda(lambda x: print(f"Debugging step output: {x}") or x)` at various points in your chain to see the exact input and output at each step. This is incredibly powerful.
      * **Tip:** For more complex debugging, LangChain's tracing (which often integrates with tools like LangSmith) can provide a visual representation of your chain's execution flow and intermediate steps. (We'll cover this in a later section\!)

-----

## Performance Considerations and Optimization Tips

Runnables and LCEL are designed with performance in mind. Here's how to leverage them for optimal application speed:

1.  **Embrace Parallelism with `RunnableParallel` and `RunnableMap`:**

      * If you have independent tasks that don't rely on each other's output, run them in parallel. `RunnableParallel` and `RunnableMap` do this automatically. This is especially impactful for multiple LLM calls, which are often the slowest part of a chain.
      * **Example:** Don't ask for a company name, wait, then ask for a slogan. Ask for both at the same time using `RunnableParallel`.

2.  **Leverage Asynchronous Operations (`astream`, `ainvoke`, `abatch`):**

      * For web applications or high-throughput services, **asynchronous operations are crucial**. They allow your program to continue doing other work while waiting for slow operations (like LLM API calls) to complete.
      * While we haven't dived deep into `asyncio` here, understand that when you're ready, Runnables provide full `async` support.

3.  **Batching with `batch()`:**

      * If you need to process multiple inputs independently (e.g., process 100 customer reviews), use the `batch()` method. LangChain can often optimize these calls by sending multiple requests to the LLM API in a single network batch, if the LLM provider supports it. This reduces overhead and latency.

4.  **Minimize LLM Calls:**

      * LLMs are powerful, but they are also the most expensive and slowest component.
      * **Be precise with prompts:** Try to get all necessary information in one LLM call if possible, rather than chaining multiple LLM calls for minor transformations.
      * **Cache results:** For frequently asked questions or stable data, consider caching LLM responses outside of LangChain to avoid redundant calls. (This is an application-level optimization).

5.  **Efficient Data Flow:**

      * Avoid sending unnecessarily large amounts of data between Runnables. Only pass what's needed for the next step.
      * Use `RunnableLambda` for efficient in-memory transformations that don't involve external calls.

-----

## Connecting to Broader RAG Architecture (A Glimpse)

While we haven't introduced Vector Databases or Retrieval yet, it's important to understand how Runnables fit into the bigger picture of a **Retrieval-Augmented Generation (RAG)** system.

In a RAG application, the general flow often looks something like this:

1.  **User Query:** The user asks a question.
2.  **Query Transformation (RunnableSequence/Lambda):** The raw query might be cleaned, expanded, or rephrased to be more effective for retrieval. (You'd use `RunnableLambda` here).
3.  **Retrieval (Runnable, eventually a Vector Store Retriever):** This is where you'd query a vector database to find relevant documents or chunks of text based on the transformed query. **This retrieval component will also be a `Runnable`\!**
4.  **Context Preparation (RunnableMap/Lambda):** The retrieved documents are combined with the original query to form a comprehensive context. (Often done with `RunnableMap` to assemble the prompt input).
5.  **Augmented Generation (Prompt | LLM | Parser):** The context and query are fed into an LLM via a prompt to generate a grounded answer. This is the part we've been building directly with `prompt | llm | parser`.
6.  **Output Processing (RunnableLambda):** The LLM's answer might be further processed, formatted, or validated before being presented to the user.

**The beautiful thing about Runnables is that *every single one of these steps* in a RAG pipeline can be implemented as a Runnable and chained together using LCEL.** This makes building, debugging, and extending RAG systems incredibly modular and powerful.

You'll see in future sections that LangChain's `VectorStoreRetriever` (which connects to vector databases) and other tools are all designed to be `Runnable`s, allowing them to fit perfectly into the workflows we're building today.

-----

## Key Takeaways

Remember these core concepts as you build with Runnables:

  * **Runnables are the standardized interface** for every component in LangChain.
  * The **`|` (pipe) operator is the core of LCEL**, allowing you to intuitively chain Runnables together.
  * **`invoke()`** for single, synchronous runs; **`stream()`** for incremental output; **`batch()`** for multiple inputs concurrently.
  * **`RunnableLambda`** for custom Python functions and data transformations.
  * **`RunnablePassthrough`** for passing inputs along, and especially **`.assign()`** for enriching inputs.
  * **`RunnableParallel`** for concurrent execution and combining results.
  * **`RunnableSequence`** (implicitly created by `|`) for linear, step-by-step workflows.
  * **`RunnableBranch`** for conditional logic and dynamic routing.
  * **`RunnableMap`** for creating structured dictionary inputs from multiple sources.
  * You can **create your own `Custom Runnables`** to integrate unique logic into your chains.
  * **Always consider input and output types** for each Runnable in your chain to avoid errors.
  * **Parallelization and async operations** are key for performance.

-----

## Exercises and Thought Experiments

To solidify your understanding, try these exercises. Don't worry about perfect code; focus on the design of the chain using Runnables.

1.  **Personalized Greeting Chain:**

      * **Goal:** Create a chain that takes a user's `name` and their `mood` as input, and generates a personalized greeting.
      * **Challenge:** Use `RunnableMap` to combine the `name` and `mood` into a dictionary for a `ChatPromptTemplate`.

2.  **Multi-Language Translator (Conceptual):**

      * **Goal:** Design a chain that takes a `text` and a `target_language`. If the `target_language` is "French", use one prompt/LLM combination, otherwise use a default one.
      * **Challenge:** Use `RunnableBranch` to implement the conditional translation logic. (You don't need actual translation, just different prompts).

3.  **Product Review Analyzer:**

      * **Goal:** Take a `product_review` text. In parallel, run two LLM calls: one to extract `sentiment` (positive/negative) and another to extract `keywords`. Combine these into a single dictionary output.
      * **Challenge:** Use `RunnableParallel` to execute the two analysis tasks concurrently.

Think about how you would structure these using the Runnables we just discussed. How would the input flow through each step? What would the output of each Runnable be?

-----

That wraps up our deep dive into Chains and Runnables\! You now have the fundamental building blocks to start creating sophisticated and highly customized LLM applications with LangChain. The power of LCEL, built upon the `Runnable` interface, will allow you to construct incredibly flexible and performant workflows.

